In [ ]:
!pip install newsapi-python

In [6]:
import pandas as pd
from newsapi import NewsApiClient
import sqlite3
import logging

In [7]:
# Initalize NewsApiClient
news_api = NewsApiClient(api_key=api_key)

In [34]:
# Create a function to get all news articles related to AI
def extract_news_data():
    try:
        news = news_api.get_everything(q="AI", language="en", sort_by="publishedAt")
        return news["articles"]
        logging.info("Connection is successful")
    except:
        logging.error("Connection unsuccessful")
        return None
    

articles = extract_news_data()
for i in range(5):
    print(articles[i])

{'source': {'id': None, 'name': 'Investing.com'}, 'author': 'Investing.com', 'title': 'Casdin Capital buys $204k of 2seventy bio shares', 'description': 'Casdin Capital buys $204k of 2seventy bio shares', 'url': 'https://www.investing.com/news/company-news/casdin-capital-buys-204k-of-2seventy-bio-shares-93CH-3357878', 'urlToImage': 'https://i-invdn-com.investing.com/news/World_News_9_800x533_L_1420026261.jpg', 'publishedAt': '2024-03-29T00:31:16Z', 'content': 'Casdin Capital, LLC, a prominent investment firm, has made a significant purchase of shares in 2seventy bio, Inc. (NASDAQ:TSVT), a biotechnology company focused on pharmaceutical preparations. The tr… [+3170 chars]'}
{'source': {'id': None, 'name': 'CNET'}, 'author': 'Ian Sherr', 'title': 'Elon Musk Makes Grok AI Available to All Paying X Subscribers: What to Know - CNET', 'description': 'The Grok AI chatbot that X has been testing is becoming more widely available.', 'url': 'https://www.cnet.com/tech/elon-musk-makes-grok-ai-avai

In [35]:
def clean_author_col(text):
    try:
        return text.split(',')[0].title()
    except AttributeError:
        return "No Author"

In [36]:
def transform_news_data(articles):
    article_list = []
    for article in articles:
        article_list.append([article["author"],article["title"],article["publishedAt"],article["content"],article["url"],article["source"]["name"]])
    articles_df = pd.DataFrame(article_list, columns=["Author", "Title", "PublishedAt", "Content", "URL", "Source"])
    articles_df["PublishedAt"] = pd.to_datetime(articles_df["PublishedAt"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    articles_df["Author"] = articles_df["Author"].apply(clean_author_col)
    return articles_df    

In [37]:
articles_df = transform_news_data(articles)
articles_df.head()

,Author,Title,PublishedAt,Content,URL,Source
0,Investing.Com,Casdin Capital buys $204k of 2seventy bio shares,2024-03-29 00:31:16,"Casdin Capital, LLC, a prominent investment fi...",https://www.investing.com/news/company-news/ca...,Investing.com
1,Ian Sherr,Elon Musk Makes Grok AI Available to All Payin...,2024-03-29 00:30:16,Elon Musk is making his company's Grok artific...,https://www.cnet.com/tech/elon-musk-makes-grok...,CNET
2,Investing.Com,Voip-Pal director Kevin Williams sells over $6...,2024-03-29 00:24:14,Voip-Pal.com Inc (OTCMKTS:VPLM) director Kevin...,https://www.investing.com/news/company-news/vo...,Investing.com
3,Pymnts.Com,White House Wants Chief AI Officers to Keep Fe...,2024-03-29 00:22:13,The White House unveiled a groundbreaking poli...,https://biztoc.com/x/e1792c3ec5142638,Biztoc.com
4,No Author,Louis Rossmann uses AI to auto-check EULAs for...,2024-03-29 00:21:17,"Pro Audio Forum - VST, AU, AAX Plugin and Audi...",https://www.kvraudio.com/forum/viewtopic.php?p...,Kvraudio.com


In [42]:
def load_news_data(data):
    con = sqlite3.connect("NewsDB")
    cur = con.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS news_table(
    "Source" VARCHAR(30), 
    "Author" TEXT, 
    "Title" TEXT, 
    "URL" TEXT, 
    "PublishedAt" TEXT, 
    "Content" TEXT
    )
    ''')
    data.to_sql(name="news_table", con=con, index=False, if_exists="append")

load_news_data(articles_df)